# Text Preprocessing 
### convert text features into numerical features ML models can work with

## 1. General text pre-processing methods

In [1]:
text = "   This is a message to be cleaned. It may involve some things like: <br>, ?, :, ''  adjacent spaces and tabs     .  "

### get rid of leading/trailing whitespace

In [6]:
text = text.strip().lower()
print(text)

this is a message to be cleaned it may involve some things like      adjacent spaces and tabs


### Remove HTML tag, markups

In [3]:
import re
text = re.compile('<.*?>').sub('', text)
print(text)

   This is a message to be cleaned. It may involve some things like: , ?, :, ''  adjacent spaces and tabs     .  


### Replace punctuation with space

In [5]:
import string
text = re.compile('[%s]' % re.escape(string.punctuation)).sub('', text)
print(text)

   This is a message to be cleaned It may involve some things like      adjacent spaces and tabs       


### Remove extra spaces and tabs

In [7]:
text = re.sub('\s+', ' ', text)
print(text)

this is a message to be cleaned it may involve some things like adjacent spaces and tabs


## 2. Lexicon-based text pre-processing
### normalize sentences in the dataset so that sentences are in a similar format

### Stop word removal

In [ ]:
stop_words = ["a", "an", "the", "this", "that", "is", "it", "to", "and"]
